# Reading data from Excel

Let's get some data. Download [Sample Superstore Sales .xls file](https://community.tableau.com/docs/DOC-1236) or and open it in Excel to see what it looks like.

Data of interest that we want to process in Python often comes in the form of an Excel spreadsheet, but the data is in a special non-text-based (binary) format that we can't read directly:

In [2]:
with open('SampleSuperstore.xls', "rb") as f:
    txt = f.read()
    print(txt[0:100])

b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00>\x00\x03\x00\xfe\xff\t\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x004\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x92\x19\x00\x00\x01\x00\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00b\x00\x00\x00\xe3\x00\x00\x00d\x01\x00\x00\xe5\x01\x00\x00f\x02\x00\x00'


## Converting Excel files with csvkit

There's a really useful tool kit called `csvkit`, which you can install with:

In [3]:
! pip install -q -U csvkit

Then, the following command works without having to run or even own Excel on your laptop (but you might get lots of warnings):

In [13]:
! in2csv SampleSuperstore.xls > SampleSuperstore.csv

In [14]:
! head -3 SampleSuperstore.csv  # show 3 lines of t.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1.0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2.0,0.0,41.9136
2.0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.9399999999999,3.0,0.0,219.58199999999997


You might be wondering why we would need a commandline version that converts Excel files to CSV.  Certainly, as we see in the next section, we can use pandas.  The difference is that using the command line means we don't have to write software, which is much more involved than simply executing a few lines on the commandline. For example, we could easily use a shell `for` loop to convert multiple Excel files to csv without starting a Python development environment.

## Reading Excel files with Pandas

The easiest way to read Excel files with Python is to use Pandas:

In [10]:
import pandas as pd
table = pd.read_excel("SampleSuperstore.xls") # XLS NOT CSV!
table.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


# Reading CSV data using built-in csv package

Grab the CSV version of the Excel file [SampleSuperstoreSales.csv](https://github.com/parrt/msan692/raw/master/data/SampleSuperstoreSales.csv) we've been playing with. You already know how to load CSV with pandas, but sometimes it's reasonable to process the CSV at a lower level, without getting a big data frame back. For example, we might want to simply process the CSV file line by line if it is truly huge.

In your project, you dealt with very simple CSV, but it can get much more complicated so let's take a quick look how to use the `csv` package. CSV files can get complicated when we need to embed a comma. One such case from the above file shows how fields with commas get quoted:

```
"Eldon Base for stackable storage shelf, platinum"
```

What happens when we want to encode a quote? Well, somehow people decided that `""` double quotes was the answer (not!) and we get fields encoded like this:

```
"1.7 Cubic Foot Compact ""Cube"" Office Refrigerators"
```

The good news is that Python's `csv` package knows how to read Excel-generated files that use such encoding. Here's a sample script that reads such a file into a list of lists:

In [15]:
import sys
import csv

table_file = "SampleSuperstore.csv"
with open(table_file, "r") as csvfile:
    f = csv.reader(csvfile, dialect='excel')
    data = []
    for row in f:
        data.append(row)

print(data[:6])

[['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'], ['1.0', 'CA-2016-152156', '2016-11-08', '2016-11-11', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', '42420.0', 'South', 'FUR-BO-10001798', 'Furniture', 'Bookcases', 'Bush Somerset Collection Bookcase', '261.96', '2.0', '0.0', '41.9136'], ['2.0', 'CA-2016-152156', '2016-11-08', '2016-11-11', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', '42420.0', 'South', 'FUR-CH-10000454', 'Furniture', 'Chairs', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', '731.9399999999999', '3.0', '0.0', '219.58199999999997'], ['3.0', 'CA-2016-138688', '2016-06-12', '2016-06-16', 'Second Class', 'DV-13045', 'Darrin Van Huff', 'Corporate', '

Or add to a numpy `array`:

In [16]:
import numpy as np
np.array(data)

array([['Row ID', 'Order ID', 'Order Date', ..., 'Quantity', 'Discount',
        'Profit'],
       ['1.0', 'CA-2016-152156', '2016-11-08', ..., '2.0', '0.0',
        '41.9136'],
       ['2.0', 'CA-2016-152156', '2016-11-08', ..., '3.0', '0.0',
        '219.58199999999997'],
       ...,
       ['9992.0', 'CA-2017-121258', '2017-02-26', ..., '2.0', '0.2',
        '19.393200000000007'],
       ['9993.0', 'CA-2017-121258', '2017-02-26', ..., '4.0', '0.0',
        '13.32'],
       ['9994.0', 'CA-2017-119914', '2017-05-04', ..., '2.0', '0.0',
        '72.94799999999998']], dtype='<U127')

## Reading CSV into Pandas Data frames

*This should be review for you...*

In the end, the easiest way to deal with loading CSV files is probably with [Pandas](http://pandas.pydata.org/), even if you need the data in a different data structure. For example, to load our sales CSV, we don't even have to manually open and close a file:

In [18]:
import pandas
df = pandas.read_csv("SampleSuperstore.csv")
df.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1.0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2.0,0.0,41.9136
1,2.0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3.0,0.0,219.5820
2,3.0,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2.0,0.0,6.8714


Pandas hides all of the details. I also find that pulling out columns is nice with pandas. Here's how to print the customer name column:

In [19]:
df['Customer Name'].head()

0        Claire Gute
1        Claire Gute
2    Darrin Van Huff
3     Sean O'Donnell
4     Sean O'Donnell
Name: Customer Name, dtype: object

In [20]:
df.Profit.head()

0     41.9136
1    219.5820
2      6.8714
3   -383.0310
4      2.5164
Name: Profit, dtype: float64